In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.base import clone
import lightgbm as lgb
from tqdm import tqdm

import xgboost as xgb
from sklearn.model_selection import train_test_split


train_path = "C:\\Users\\USER\\Desktop\\House price\\dataset.csv"
test_path = "C:\\Users\\USER\\Desktop\\House price\\test.csv"
sam_path = "C:\\Users\\USER\\Desktop\\House price\\sample_submission.csv"

print("Libraries installed successfully!")

Libraries installed successfully!


import data

In [2]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)
sample_df = pd.read_csv(sam_path)

print(train_df.shape)
print(train_df.columns)
train_df.head()

(200000, 47)
Index(['id', 'sale_date', 'sale_price', 'sale_nbr', 'sale_warning',
       'join_status', 'join_year', 'latitude', 'longitude', 'area', 'city',
       'zoning', 'subdivision', 'present_use', 'land_val', 'imp_val',
       'year_built', 'year_reno', 'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
       'grade', 'fbsmt_grade', 'condition', 'stories', 'beds', 'bath_full',
       'bath_3qtr', 'bath_half', 'garb_sqft', 'gara_sqft', 'wfnt', 'golf',
       'greenbelt', 'noise_traffic', 'view_rainier', 'view_olympics',
       'view_cascades', 'view_territorial', 'view_skyline', 'view_sound',
       'view_lakewash', 'view_lakesamm', 'view_otherwater', 'view_other',
       'submarket'],
      dtype='object')


,id,sale_date,sale_price,sale_nbr,sale_warning,join_status,join_year,latitude,longitude,area,...,view_olympics,view_cascades,view_territorial,view_skyline,view_sound,view_lakewash,view_lakesamm,view_otherwater,view_other,submarket
0,0,2014-11-15,236000,2.0,,nochg,2025,47.2917,-122.3658,53,...,0,0,0,0,0,0,0,0,0,I
1,1,1999-01-15,313300,NaN,26,nochg,2025,47.6531,-122.1996,74,...,0,0,0,0,0,1,0,0,0,Q
2,2,2006-08-15,341000,1.0,,nochg,2025,47.4733,-122.1901,30,...,0,0,0,0,0,0,0,0,0,K
3,3,1999-12-15,267000,1.0,,nochg,2025,47.4739,-122.3295,96,...,0,0,0,0,0,0,0,0,0,G
4,4,2018-07-15,1650000,2.0,,miss99,2025,47.7516,-122.1222,36,...,0,0,0,0,0,0,0,0,0,P


In [3]:
#計算缺失值
missing_values = train_df.isnull().sum()
missing_values[missing_values > 0].sort_values(ascending=False)

sale_nbr       42182
subdivision    17550
submarket       1717
dtype: int64

feature engineering

In [ ]:
# 建立 train_encoded
processed_cols = []
train_encoded = pd.DataFrame()

# 一、One-hot 編碼處理欄位
confirmed_one_hot = [
    'join_status', 'condition', 'stories', 'grade', 'fbsmt_grade', 'present_use'
]
onehot_df = pd.get_dummies(train_df[confirmed_one_hot], drop_first=False)
train_encoded = pd.concat([train_encoded, onehot_df], axis=1)
processed_cols += confirmed_one_hot

# 二、處理日期特徵 + 拆解年月與季節
train_df['sale_date'] = pd.to_datetime(train_df['sale_date'], errors='coerce')
train_encoded['sale_year'] = train_df['sale_date'].dt.year
train_encoded['sale_month'] = train_df['sale_date'].dt.month
train_encoded['sale_season'] = ((train_encoded['sale_month'] - 1) // 3 + 1)
processed_cols += ['sale_date']

# 三、原始數值直接加入
direct_add_cols = [
    'id', 'sale_price', 'join_year', 'latitude', 'longitude',
    'area', 'land_val', 'imp_val', 'year_built', 'year_reno',
    'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
    'beds', 'garb_sqft', 'gara_sqft', 'golf', 'greenbelt',

    'bath_full', 'bath_3qtr', 'bath_half', 'wfnt', 'noise_traffic',
    'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial',
    'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm',
    'view_otherwater', 'view_other'
    #'subdivision','sale_nbr'  to much missing value
]
for col in direct_add_cols:
    train_encoded[col] = train_df[col]
processed_cols += direct_add_cols

# 四、統整城市、市場與銷售警告資訊
top_cities = train_df['city'].value_counts().nlargest(10).index.tolist()
top_supermarket = train_df['submarket'].value_counts().nlargest(10).index.tolist()
top_sale_warning = train_df['sale_warning'].value_counts().nlargest(15).index.tolist()

train_encoded['city_simplified'] = train_df['city'].apply(lambda x: x if x in top_cities else 'other')
train_encoded['submarket_simplified'] = train_df['submarket'].apply(lambda x: x if x in top_supermarket else 'other')
train_encoded['sale_warning_simplified'] = train_df['sale_warning'].apply(lambda x: x if x in top_sale_warning else 'other')

city_dummy = pd.get_dummies(train_encoded['city_simplified'], prefix='city', drop_first=False)
submarket_dummy = pd.get_dummies(train_encoded['submarket_simplified'], prefix='submarket', drop_first=False)
sale_warning_dummy = pd.get_dummies(train_encoded['sale_warning_simplified'], prefix='sale_warning', drop_first=False)
train_encoded = pd.concat([train_encoded, city_dummy, submarket_dummy, sale_warning_dummy], axis=1)
processed_cols += ['city', 'submarket', 'sale_warning']

# 五、Zoning 群組分類
def zoning_group_classify(z):
    if pd.isna(z): return 'other'
    z = z.upper()
    if 'SF' in z: return 'SF'
    elif 'MR' in z: return 'MR'
    elif 'NC' in z: return 'NC'
    #elif 'HR' in z: return 'HR'
    elif 'IG' in z: return 'IG'
    elif 'P' in z: return 'P'
    return 'other'

train_encoded['zoning_group'] = train_df['zoning'].apply(zoning_group_classify)
zoning_dummy = pd.get_dummies(train_encoded['zoning_group'], prefix='zoning_group', drop_first=False)
train_encoded = pd.concat([train_encoded, zoning_dummy], axis=1)
train_encoded.drop(columns=['zoning_group'], inplace=True)
processed_cols += ['zoning']

# 六、碎片化資訊統整成新欄位
train_encoded['age'] = train_encoded['sale_year'] - train_encoded['year_built']
train_encoded['renovated'] = np.where(train_encoded['year_reno'] > 0, 1, 0)
train_encoded['years_since_reno'] = np.where(train_encoded['renovated'], train_encoded['sale_year'] - train_encoded['year_reno'], 0)
train_encoded['total_baths'] = train_encoded['bath_full'] + 0.75 * train_encoded['bath_3qtr'] + 0.5 * train_encoded['bath_half']
train_encoded['total_value'] = train_encoded['land_val'] + train_encoded['imp_val']
train_encoded['living_area'] = train_encoded['sqft'] + train_encoded['sqft_fbsmt']

# 七、刪除用完的簡化文字類欄位
for col in ['city_simplified', 'submarket_simplified', 'sale_warning_simplified']:
    train_encoded.drop(columns=[col], inplace=True)


In [5]:
direct_add_cols = [
    'id', 'join_year', 'latitude', 'longitude',
    'area', 'land_val', 'imp_val', 'year_built', 'year_reno',
    'sqft_lot', 'sqft', 'sqft_1', 'sqft_fbsmt',
    'beds', 'garb_sqft', 'gara_sqft', 'golf', 'greenbelt',

    'bath_full', 'bath_3qtr', 'bath_half',
    'wfnt', 'noise_traffic',
    'view_rainier', 'view_olympics', 'view_cascades', 'view_territorial',
    'view_skyline', 'view_sound', 'view_lakewash', 'view_lakesamm',
    'view_otherwater', 'view_other'
    #'subdivision','sale_nbr'沒有做這個 用意不大
]

In [6]:
# 建立 test_encoded 空表
test_encoded = pd.DataFrame()

# 1. One-hot 欄位
test_onehot_df = pd.get_dummies(test_df[confirmed_one_hot], drop_first=False)
test_encoded = pd.concat([test_encoded, test_onehot_df], axis=1)

# 2. 日期處理
test_encoded['sale_date'] = pd.to_datetime(test_df['sale_date'], errors='coerce')
test_encoded['sale_year'] = test_encoded['sale_date'].dt.year
test_encoded['sale_month'] = test_encoded['sale_date'].dt.month
test_encoded['sale_season'] = ((test_encoded['sale_month'] - 1) // 3 + 1)
test_encoded.drop(columns=['sale_date'], inplace=True)

# 3. 加入 direct_add_cols 欄位
for col in direct_add_cols:
    test_encoded[col] = test_df[col]

# 4. city / submarket / sale_warning (simplified)
test_encoded['city_simplified'] = test_df['city'].apply(lambda x: x if x in top_cities else 'other')
city_dummy = pd.get_dummies(test_encoded['city_simplified'], prefix='city', drop_first=False)
test_encoded = pd.concat([test_encoded, city_dummy], axis=1)

test_encoded['submarket_simplified'] = test_df['submarket'].apply(lambda x: x if x in top_supermarket else 'other')
submarket_dummy = pd.get_dummies(test_encoded['submarket_simplified'], prefix='submarket', drop_first=False)
test_encoded = pd.concat([test_encoded, submarket_dummy], axis=1)

test_encoded['sale_warning_simplified'] = test_df['sale_warning'].apply(lambda x: x if x in top_sale_warning else 'other')
sale_warning_dummy = pd.get_dummies(test_encoded['sale_warning_simplified'], prefix='sale_warning', drop_first=False)
test_encoded = pd.concat([test_encoded, sale_warning_dummy], axis=1)

# 5. Zoning 分群 One-hot
test_encoded['zoning_group'] = test_df['zoning'].apply(zoning_group_classify)
zoning_dummy = pd.get_dummies(test_encoded['zoning_group'], prefix='zoning_group', drop_first=False)
test_encoded = pd.concat([test_encoded, zoning_dummy], axis=1)
test_encoded.drop(columns=['zoning_group', 'city_simplified', 'submarket_simplified', 'sale_warning_simplified'], inplace=True)


#碎片化資訊統整成新欄位
test_encoded['age'] = test_encoded['sale_year'] - test_encoded['year_built']
test_encoded['renovated'] = np.where(test_encoded['year_reno'] > 0, 1, 0)
test_encoded['years_since_reno'] = np.where(test_encoded['renovated'], test_encoded['sale_year'] - test_encoded['year_reno'], 0)
test_encoded['total_baths'] = test_encoded['bath_full'] + 0.75 * test_encoded['bath_3qtr'] + 0.5 * test_encoded['bath_half']
test_encoded['total_value'] = test_encoded['land_val'] + test_encoded['imp_val']
test_encoded['living_area'] = test_encoded['sqft'] + test_encoded['sqft_fbsmt']

In [7]:
#確認資料類型
train_encoded.dtypes.value_counts()

bool       52
int64      40
float64     4
int32       4
Name: count, dtype: int64

model(XGBoost)

In [ ]:
# 分割訓練特徵與目標
X = train_encoded.drop(columns=['sale_price', 'id'])  # id 可留給最後輸出
y = train_encoded['sale_price']

In [ ]:

# 建立上下界
y_lower = y * 0.9
y_upper = y * 1.1

# 拆分資料
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
y_lower_train = y_train * 0.9
y_lower_val = y_val * 0.9
y_upper_train = y_train * 1.1
y_upper_val = y_val * 1.1
# 模型 A：下限
model_lower = xgb.XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.01,early_stopping_rounds=20,eval_metric='rmse',tree_method='gpu_hist')

# 模型 B：上限
model_upper = xgb.XGBRegressor(n_estimators=500, max_depth=6, learning_rate=0.01,early_stopping_rounds=20,eval_metric='rmse',tree_method='gpu_hist')


In [ ]:
#下限預測
model_lower.fit(X=X_train,
                y=y_lower_train,
                eval_set=[(X_val, y_lower_val)],
                verbose=True)


In [ ]:
#上限預測
model_upper.fit(X=X_train,
                y=y_upper_train,
                eval_set=[(X_val, y_upper_val)],
                verbose=True)

LGBM Quantile

In [8]:
# 分割訓練特徵與目標
X = train_encoded.drop(columns=['sale_price', 'id'])  # id 可留給最後輸出
y = train_encoded['sale_price']

In [9]:
def winkler_score(y_true, lower, upper, alpha=0.1):
    width = upper - lower
    below = np.maximum(lower - y_true, 0)
    above = np.maximum(y_true - upper, 0)
    return width + (2 / alpha) * (below + above)

In [18]:
def oof_and_hill_climb_two_weights(X, y, model_lower, model_upper, alpha=0.1, n_splits=5, seed=42, steps=100):
    oof_lowers = np.zeros(len(X))
    oof_uppers = np.zeros(len(X))

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train = y.iloc[train_idx]

        lower_model = clone(model_lower)
        upper_model = clone(model_upper)

        lower_model.fit(X_train, y_train-0.01)
        upper_model.fit(X_train, y_train+0.01)

        oof_lowers[val_idx] = lower_model.predict(X_val)
        oof_uppers[val_idx] = upper_model.predict(X_val)

    # 初始化雙權重
    current_w1 = 0.425  # 下限 weight
    current_w2 = 0.575  # 上限 weight

    best_score = np.inf
    best_weights = (current_w1, current_w2)

    for step in range(steps):
        # 微調 perturbation，讓 weight 有隨機性（避免卡住）
        perturb1 = np.random.dirichlet([9])[0] - 0.9
        perturb2 = np.random.dirichlet([9])[0] - 0.9

        w1 = np.clip(current_w1 + 0.1 * perturb1, 0, 1)
        w2 = np.clip(current_w2 + 0.1 * perturb2, 0, 1)

        # 雙權重組合
        lower_combined = w1 * oof_lowers + (1 - w1) * oof_uppers
        upper_combined = w2 * oof_uppers + (1 - w2) * oof_lowers

        # 修正：確保上下限方向正確（防止預測範圍錯位）
        lower_combined, upper_combined = np.minimum(lower_combined, upper_combined), np.maximum(lower_combined, upper_combined)

        score = np.mean(winkler_score(y, lower_combined, upper_combined, alpha))

        if score < best_score:
            best_score = score
            best_weights = (w1, w2)
            current_w1, current_w2 = w1, w2
            print(f"[Step {step}] ✅ Improved Score: {best_score:.2f} (w1: {w1:.4f}, w2: {w2:.4f})")

    return oof_lowers, oof_uppers, best_weights, best_score

In [19]:
models = {
    "lower": lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.05,
        device="cpu",
        n_estimators=1500,
        learning_rate=0.05,
        num_leaves=63,
        subsample=0.8,
        subsample_freq=1,
        random_state=42
    ),
    "upper": lgb.LGBMRegressor(
        objective="quantile",
        alpha=0.95,
        device="cpu",
        n_estimators=1500,
        learning_rate=0.05,
        num_leaves=63,
        subsample=0.8,
        subsample_freq=1,
        random_state=42
    )
}

In [20]:
oof_lowers, oof_uppers, (w1, w2), best_score = oof_and_hill_climb_two_weights(
    X, y,
    model_lower=models["lower"],
    model_upper=models["upper"],
    alpha=0.1, 
    n_splits=5,
    steps=100
)

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.018651 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3845
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 97
[LightGBM] [Info] Start training from score 184999.984375
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016148 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3845
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 97
[LightGBM] [Info] Start training from score 1435000.000000
[LightGBM] [War

輸出test xgboost

In [ ]:
#填補缺漏欄位（對齊訓練集欄位）
missing_cols = set(X_train.columns) - set(test_encoded.columns)
for col in missing_cols:
    test_encoded[col] = 0

# 確保欄位順序一致
test_encoded = test_encoded[X_train.columns]

In [169]:
print(set(X_train.columns) - set(test_encoded.columns))

set()


In [146]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df.head()

,id,pi_lower,pi_upper
0,200000,0,100000000
1,200001,0,100000000
2,200002,0,100000000
3,200003,0,100000000
4,200004,0,100000000


In [171]:
test_encoded['id'] = test_df['id']  # 這行先補上 id

In [ ]:
# 分割訓練特徵與目標
X = train_encoded.drop(columns=['sale_price', 'id'])  # id 可留給最後輸出
y = train_encoded['sale_price']

In [ ]:
# 預測上下限
y_lower_pred = model_lower.predict(test_encoded.drop(columns=['id']))
y_upper_pred = model_upper.predict(test_encoded.drop(columns=['id']))

# 建立提交檔
submission_df = pd.DataFrame({
    'id': test_encoded['id'],  # 必須與 sample_submission 對齊
    'pi_lower': y_lower_pred,
    'pi_upper': y_upper_pred
})

# 輸出成 CSV
submission_df.to_csv('xgb_predict.csv', index=False)
print(submission_df.head())

輸出test LGBM

In [15]:
#填補缺漏欄位（對齊訓練集欄位）
missing_cols = set(X.columns) - set(test_encoded.columns)
for col in missing_cols:
    test_encoded[col] = 0

# 確保欄位順序一致
test_encoded = test_encoded[X.columns]

In [ ]:
model_lower = models["lower"]
model_upper = models["upper"]

final_model_lower = clone(model_lower).fit(X, y)
final_model_upper = clone(model_upper).fit(X, y)

'''test_lower = final_model_lower.predict(test_encoded)
test_upper = final_model_upper.predict(test_encoded)

final_lower = best_weight * test_lower
final_upper = best_weight * test_upper

final_lower, final_upper = np.minimum(final_lower, final_upper), np.maximum(final_lower, final_upper)
final_lower = np.maximum(final_lower, 0)'''


test_lower = final_model_lower.predict(test_encoded)
test_upper = final_model_upper.predict(test_encoded)

final_lower = w1 * test_lower + (1 - w1) * test_upper
final_upper = w2 * test_upper + (1 - w2) * test_lower

final_lower, final_upper = np.minimum(final_lower, final_upper), np.maximum(final_lower, final_upper)
final_lower = np.maximum(final_lower, 0)  # optional

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3845
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 97
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 630, Vendor: Intel(R) Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 27 dense feature groups (4.27 MB) transferred to GPU in 0.005452 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 185000.000000
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 3845
[LightGBM] [Info] Number of data points in the train set: 160000, number of used features: 97
[LightGBM] [Info] Using GPU Device: Intel(R) UHD Graphics 630, Vendor: Intel(R

In [21]:
submission_df = pd.read_csv('sample_submission.csv')
submission_df.head()
test_encoded['id'] = test_df['id']  # 這行先補上 id

In [22]:
submission_df = pd.DataFrame({
    'id': test_encoded['id'],  # 必須與 sample_submission 對齊
    'pi_lower': final_lower,
    'pi_upper': final_upper
})

# 輸出成 CSV
submission_df.to_csv('lgbm_predict.csv', index=False)
print(submission_df.head())

       id       pi_lower      pi_upper
0  200000  823500.777056  1.125383e+06
1  200001  590158.900174  7.270068e+05
2  200002  469229.953050  6.502398e+05
3  200003  315274.697956  4.363184e+05
4  200004  413919.532224  6.498657e+05
